In [6]:
import src.config as config

In [7]:
from datetime import datetime, timedelta
import pandas as pd

# extract the current date and round it to the nearest hour
current_date = pd.to_datetime(datetime.utcnow()).floor('h')
print(f'current_date: {current_date}')

# range of historical data to fetch
# fetch data from the last 28 days to add redundancy to a pipeline as failures can happen
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

current_date: 2023-05-28 05:00:00


In [12]:
# simulate a call to a datawarehouse to fetch historical data
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # download 2 files from website
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [13]:
# calling a function of data simulation
rides = fetch_batch_raw_data(from_date = fetch_data_from, to_date=fetch_data_to)

from_date=Timestamp('2023-04-30 05:00:00'), to_date_=Timestamp('2022-05-29 05:00:00')
File 2022-05 was already in local storage
File 2022-05 was already in local storage


In [14]:
# transform the raw data into ts data
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:00<00:00, 637.74it/s]


In [15]:
# connect to the feature store and the feature group
import hopsworks

# connect to hopsworks
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY,
)

# connect to feature store
feature_store = project.get_feature_store()

# connect to feature group
feature_group = feature_store.get_or_create_feature_group(
    config.FEATURE_GROUP_NAME, 
    config.FEATURE_GROUP_VERSION,
    description='Time series data at hourly frequency',
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
    )

/Users/nick/taxi_demand_predictor/.venv/lib/python3.9/site-packages/great_expectations/compatibility/sqlalchemy.py:20: UserWarning: SQLAlchemy v2.0.0 or later is not yet supported by Great Expectations.
  warnings.warn(


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/50351
Connected. Call `.close()` to terminate connection gracefully.


In [16]:
# save the data into the feature store
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 0.00% |          | Rows 0/197160 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/50351/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x163123370>, None)